In [37]:
import numpy as np
import pandas as pd

## 数据预处理

In [38]:
# 1. 读取数据
path_str = 'TrainData\Vehicle No.4.csv'
df_train_no_0 = pd.read_csv(path_str)

In [39]:
df_train_no_0.describe()

,time,speed,total_voltage,total_current,soc,temp_max,temp_min,motor_voltage,motor_current,mileage
count,3.484280e+05,348428.000000,348428.000000,348428.000000,348428.000000,348428.000000,348428.000000,348428.000000,348428.000000,348428.000000
mean,2.018049e+13,13.311358,451.933869,8.743907,55.801006,21.338423,16.916600,356.897910,11.155553,47658.416367
std,3.671734e+08,17.382670,193.484421,43.813073,30.304856,10.794129,9.667592,250.378129,44.199362,11855.682940
min,2.018010e+13,0.000000,0.000000,-163.000000,0.000000,0.000000,-1.000000,0.000000,-152.000000,33842.000000
25%,2.018012e+13,0.000000,526.900000,-10.000000,39.000000,19.000000,13.000000,0.000000,0.000000,36060.175000
50%,2.018042e+13,0.000000,532.300000,0.000000,61.000000,22.000000,16.000000,530.000000,0.000000,45219.550000
75%,2.018072e+13,31.000000,537.400000,13.000000,80.000000,29.000000,21.000000,535.000000,1.000000,55496.225000
max,2.018113e+13,58.000000,585.000000,264.000000,100.000000,42.000000,39.000000,578.000000,278.000000,69233.800000


In [40]:

# 2. 按时间排序
df_train_no_0_sorted = df_train_no_0.sort_values(by='time')

# 3 按时间片 分成不同的df 存放，处理后合并
df_train_no_0_sorted_time1 = df_train_no_0_sorted[df_train_no_0_sorted.time < 20180200000000]

df_train_no_0_sorted_time4 = df_train_no_0_sorted[(df_train_no_0_sorted.time > 20180200000000) & (df_train_no_0_sorted.time < 20180500000000)]

df_train_no_0_sorted_time7 = df_train_no_0_sorted[(df_train_no_0_sorted.time > 20180500000000) & (df_train_no_0_sorted.time < 20181000000000)]

df_train_no_0_sorted_time11 = df_train_no_0_sorted[(df_train_no_0_sorted.time > 20181000000000) & (df_train_no_0_sorted.time < 20181200000000)]

In [41]:
## 制作label列，便于之后的数据清醒，封装成函数
def data_proprecessing(df_train):
    time1_array = df_train.values
    # 增加一列作为特征列
    new_col = np.zeros((time1_array.shape[0],))
    time1_array = np.c_[time1_array, new_col]
    for i in range(time1_array.shape[0]):
        if i  == 0:
            time1_array[i][10] = 0
        else:
            time1_array[i][10] = time1_array[i][-2] - time1_array[i-1][-2]
    
    return time1_array
    
    

In [42]:
train_XY1 = data_proprecessing(df_train_no_0_sorted_time1)
train_XY4 = data_proprecessing(df_train_no_0_sorted_time4)
train_XY7 = data_proprecessing(df_train_no_0_sorted_time7)
train_XY11 = data_proprecessing(df_train_no_0_sorted_time11)

In [43]:
print(train_XY1.shape)
print(train_XY4.shape)
print(train_XY7.shape)
print(train_XY11.shape)

(131393, 11)
(76043, 11)
(79532, 11)
(61460, 11)


In [44]:
# 将各个月份的数据合并在一起
train_XY_finnal = np.vstack((train_XY1, train_XY4, train_XY7, train_XY11))
print(train_XY_finnal.shape)


(348428, 11)


In [45]:
## 保存成文件
train_XY_df = pd.DataFrame(train_XY_finnal)

train_XY_df.columns = ['time','speed','total_voltage', 'total_current', 'soc', 'temp_max', 'temp_min', 'motor_voltage', 'motor_current','mileage','label']

train_XY_df.to_csv('temp/train_XY_No4.csv', index=False)
